## Programming part of Homework 4 (Data Structures, Fall 2019)

## Name:林岳徵
## Student ID Number:108054326

### Programming problem 
This exercise is about to implement a ***skip list** using *linked list structure* with **Python**. Sample definitions for the list node and skip list classes are given as below. Recall the discussion in class. Each node has an entry with two attributes, *key* and *name*, where key is an integer and name is a string, as well as four links: *prev*, *next*, *up*, and *down*. In order to manage multiple lists used in the skip list, we use the built-in data structure, ***list***, of Python. The methods in the implemented skip list include:
1. `addEmptyList()`: This is to pad the skip list when the number of copies of the inserted node is more than the height (number of lists used) of the current skip list when inserting nodes;
2. `search(v)`: This method searches the skip list with the given node $v$ using the key;
3. `delete(v)`: This method will delete the given node $v$ from the skip list;
4. `insert(v)`: This is to insert the given node $v$ to the skip list.
The Python program starts with function `create\_SkipLists()` which reads the input file, `inFile.txt`. In the input file, each line contains only one entry: key and string and as follows:

```
10 mary
25 john
35 mars
50 lowe
```

An example of input file will also be provided.

In [28]:
import re
import random

# Two special values for boundary nodes
PLUS_INF = 99999
MINUS_INF = -99999

# Node class definition: a quadraic node having four links  
class SLnode:
    def __init__(self, key, item):
        self.key = key
        self.item = item
        self.up = None
        self.down = None
        self.next = None
        self.prev = None

    def getKey(self):
        return self.key

    def getItem(self):
        return self.item

    def getNext(self):
        return self.next

    def getPrev(self):
        return self.prev

    def getUp(self):
        return self.up

    def getDown(self):
        return self.down

    def hasNext(self):
        return (self.next!=None)

    def hasPrev(self):
        return (self.prev!=None)

    def setKey(self, key):
        self.key = key

    def setItem(self, item):
        self.item = item

    def setNext(self, p):
        self.next = p

    def setPrev(self, p):
        self.prev =p
 
    def setUp(self, p):
        self.up = p

    def setDown(self, p):
        self.down = p

# List class definition used in the skip list 
class SLlist:
    def __init__(self):
        self.leftDummy=SLnode(MINUS_INF,"")
        self.rightDummy=SLnode(PLUS_INF,"")
        self.leftDummy.setNext(self.rightDummy)
        self.rightDummy.setPrev(self.leftDummy)
        self.size = 0
        self.insert_cursor = self.getleftDummy()

    def getleftDummy(self):
        return self.leftDummy

    def getrightDummy(self):
        return self.rightDummy

    def getSize(self):
        return self.size

    def increaseSize(self):
        self.size=self.size + 1

    def decreaseSize(self):
        self.size=self.size - 1

    '''
    method insertAfter(self, p, SLnode): insert a node in the list after node p
    '''
    def insertAfter(self, p, SLnode):
        Current = self.getleftDummy()
        while Current != self.getrightDummy():
            if Current is not p:#not p 但是跑到最後會有問題
                Current = Current.getNext()
            else:
                SLnode.setNext(Current.getNext())
                Current.getNext().setPrev(SLnode)
                Current.setNext(SLnode)
                SLnode.setPrev(Current)
                self.increaseSize()
                break
            
    '''
    method print_List(self): print the content of the list
    '''
    def print_List(self):
        Current = self.getleftDummy()
        for i in range(self.getSize()+1):
            print("(",Current.key,",",Current.item,")", end = "")
            Current = Current.getNext()
        print("(",Current.key,",",Current.item,")")

# Skip list definition: a list of lists is used
class Skip_Lists:
    def __init__(self):
        self.S=[SLlist()]

    def getLists(self):
        return self.S

    # use the number of nodes in the bottom list to denote the Size
    def getSize(self):
        return self.S[0].getSize()

    # use Height to denote the number of lists used in the skip list
    def getHeight(self):
        return len(self.S)

    def isEmpty(self):
        return ((self.getHeight()==1) and (self.getSize()==0))

    # Derive the top list in the skip list
    def getTopList(self):
        return self.S[self.getHeight()-1]

    '''
    method getTopleft(self): get the topleft node in the skip list
    '''
    # Derive the topleft node in the skip list
    def getTopleft(self):
        return self.getTopList().getleftDummy()
        
    '''
    method addEmptyList(): padding the skip list when the number of copies of the inserted node
    is more than the height of the current  skip list
    '''
    def addEmptyList(self):
        self.S.append(SLlist())
        for i in range(len(self.S)-1):
            self.S[i].getleftDummy().setUp(self.S[i+1].getleftDummy())
            self.S[i].getrightDummy().setUp(self.S[i+1].getrightDummy())
            self.S[i+1].getleftDummy().setDown(self.S[i].getleftDummy())
            self.S[i+1].getrightDummy().setDown(self.S[i].getrightDummy())
        
    '''
    method search(node): search the skip list with the given node using the key
    '''
    def search(self, node):
        Current = self.getTopleft()
        Height = self.getHeight()

        while Current.getNext().getKey() > node.getKey() and Height > 1:
            Current = Current.getDown()
            Height -= 1
           
        while Current.getNext().getKey() < node.getKey() and Current.getNext() != self.S[Height-1].getrightDummy():
            Current = Current.getNext()
            
        if node.getKey() == Current.getNext().getKey():
            return [Current.getNext(),Height]
        else:
            return [None,None]

    '''
    method delete(node): delete the given node from the skip list
    '''
    def delete(self, node):
        [Current,Height] = self.search(node)
        if Current is None:
            print("Key not found in the skip lists and will not perform the deletion.")
        else:
            while Current.getDown() is not None:
                Current.getPrev().setNext(Current.getNext())
                Current.getNext().setPrev(Current.getPrev())
                if Current.getPrev() == self.S[Height - 1].getleftDummy() and Current.getNext() == self.S[Height - 1].getrightDummy():
                    self.S.pop()
                Current = Current.getDown()
                self.S[Height - 1].decreaseSize()
                Hight -= 1
            Current.getPrev().setNext(Current.getNext())
            Current.getNext().setPrev(Current.getPrev())
            self.S[Height-1].decreaseSize()#要加link list size - 1

            
        
    '''
    method insert(node): insert the given node to the skip list
    '''
    def insert(self, node):
        i = coin_tossing()
        #i = 1
        #print(node.getKey())
        [Search,Height] = self.search(node)
        if Search is not None:
            print("Key found in the skip lists and will not insert the new node.")
            return
        
        if i > self.getHeight() - 1:
            Diff = i - self.getHeight() + 1
            for j in range(Diff):
                self.addEmptyList()
            k = i
            for k in range(i,-1,-1):
                Current = self.S[k].getleftDummy()
                while Current.getNext().getKey() < node.getKey():
                    Current = Current.getNext()
                self.S[k].insertAfter(Current, node)
            return
        else:
            k = i
            for k in range(i, -1, -1):
        ##這一行以下動作是對的
                Current = self.S[k].getleftDummy()
                while Current.getNext().getKey() < node.getKey():
                    Current = Current.getNext()
                self.S[k].insertAfter(Current, node)
                return
'''
function for coin tossing with the number of heads returned
'''
def coin_tossing():
    i = 0
    while random.randint(0,1) == 0:#We use 0 represent the head of coin
        i += 1
    return i
    
'''
function for reading lines (entries) in the input text file into a list of strings
'''
def read_lines():
    f = open("inFile.txt",'r')
    line = f.read()
    line = line.split()
    f.close()
    return line
    
'''
function for starting the task
'''
def create_SkipLists():
    #
    # read the input information from the default input text file into an
    # entry list, entry_list
    #
    entry_list = read_lines()
    #
    # initiating a skip list object SL
    #
    SL = Skip_Lists()
    pairs = [0, 0]

    for index in range(0, len(entry_list)//2):
        # splitting the string by " " symbol for deriving the entry                                      
        #pairs = re.split(" ",entry_list)[index]#Original code
        pairs[0] = entry_list[2*index]
        pairs[1] = entry_list[2*index + 1]
        
        # making a new node for the entry
        newnode = SLnode(int(pairs[0]), pairs[1])
        # inserting the new node to the skip list SL
        SL.insert(newnode)

    #--------------dynamic operations with result printed -----------------------------------
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("Insert (88, luke)")
    SL.insert(SLnode(88, "luke"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (40, kite)")
    SL.delete(SLnode(40, "kite"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("Insert (27, eric)")
    SL.insert(SLnode(27, "eric"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (45, lisa)")
    SL.delete(SLnode(45, "lisa"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (27, luis)")
    SL.delete(SLnode(27, "luis"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (8, kids)")
    SL.delete(SLnode(8, "kids"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    print("delete (88, luke)")
    SL.delete(SLnode(88, "luke"))
    for i in range(0,SL.getHeight()):
        SL.S[i].print_List()

    return

print(read_lines())
print('-----------------------------------------------')
create_SkipLists()

['27', 'luis', '8', 'kids', '40', 'kite']
-----------------------------------------------
( -99999 ,  )( 8 , kids )( 27 , luis )( 40 , kite )( 40 , kite )
( -99999 ,  )( 27 , luis )( 40 , kite )( 40 , kite )
( -99999 ,  )( 40 , kite )( 40 , kite )
( -99999 ,  )( 40 , kite )( 40 , kite )
Insert (88, luke)


KeyboardInterrupt: 